In [2]:
!pip install langchain langchain-google-genai duckduckgo-search youtube_search wikipedia langchainhub langchain_experimental

import os
import getpass

from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub, LLMMathChain
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

   ---------------------------------------- 0.0/202.7 kB ? eta -:--:--
   -------- ------------------------------- 41.0/202.7 kB 2.0 MB/s eta 0:00:01
   ------------- ------------------------- 71.7/202.7 kB 787.7 kB/s eta 0:00:01
   ------------- ------------------------- 71.7/202.7 kB 787.7 kB/s eta 0:00:01
   ---------------------------- --------- 153.6/202.7 kB 766.6 kB/s eta 0:00:01
   -------------------------------------- 202.7/202.7 kB 948.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.5 MB 2.6 MB/s eta 0:00:01
   - -------------------------------------- 0.1/2.5 MB 2.4 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.5 MB 1.7 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.5 MB 1.5 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.5 MB 1.7 MB/s eta 0:00:02
   ------- -------------------------------- 0.5/2.5 MB 1.8 MB/s eta 0:00:02
   --

d:\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")

In [4]:
# setup model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.3,
    max_tokens= 200,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

In [6]:
ddg_search = DuckDuckGoSearchAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="Useful to browse information from the internet to know recent results and information you don't know. Then, tell user the result.",
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about very simple math. This tool is only for math questions and nothing else. Only input math expressions. Not supporting symbolic math.",
    ),
]

In [7]:
agent_prompt = hub.pull("mikechan/gemini")
agent_prompt.template

'Your name is “Kasper”. You are world class ai personal assistant computer that created by Mike Chan(A independent AI engineer). You are highly advanced intelligent. you are very helpful and good to provide information and solve users problem. Kasper is a AELM model (Auto Execution Language Model).\n\n“ Kasper” is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, “ Kasper” is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\n“ Kasper” is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, “ Kasper” is

In [8]:
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [9]:
memory = ConversationBufferMemory(memory_key="chat_history")


In [10]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [12]:
agent_executor.invoke({"input": "Who is roaring kitty"})["output"]



> Entering new AgentExecutor chain...


d:\Langchain\venv\lib\site-packages\langchain_google_genai\chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? Yes
Action: DuckDuckGo Search
Action Input: Who is roaring kittyRoaring Kitty, real name Keith Gill, is a trader and YouTuber who influenced millions to buy and hold GameStop stock in January 2021. He returned to X after a hiatus and caused GameStop's stock to jump 50% on Monday. Roaring Kitty, also known as Keith Gill, is a Reddit and YouTube user who bet on GameStop and inspired retail traders to squeeze short-sellers. He holds 5 million shares and 120,000 call options of GameStop, worth over $289 million, and could become a billionaire if the stock rises. Keith Gill is back. The famed meme stock icon, also known as "Roaring Kitty," made a long-awaited appearance on a livestream on his YouTube channel Friday. Sporting long hair, a thick bandage ... Keith Gill is a former financial analyst who became famous for investing in GameStop stock and sharing his views on social media. He resigned from MassMutual in 2021 and testified before Congress about his

d:\Langchain\venv\lib\site-packages\langchain_google_genai\chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Do I need to use a tool? No
Final Answer: Roaring Kitty is the online alias of Keith Gill, a financial analyst who became famous for investing in GameStop stock and sharing his views on social media. He resigned from MassMutual in 2021 and testified before Congress about his role in the GameStop saga.

> Finished chain.


'Roaring Kitty is the online alias of Keith Gill, a financial analyst who became famous for investing in GameStop stock and sharing his views on social media. He resigned from MassMutual in 2021 and testified before Congress about his role in the GameStop saga.'